In [1]:
!pip install -q langgraph langchain-core langchain-anthropic anthropic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 453.7/453.7 kB 12.2 MB/s eta 0:00:00


In [2]:
# Cell 2: load API key from Colab Secrets (recommended)
import os
from google.colab import userdata

CANDIDATES = ["ANTHROPIC_API_KEY", "ANTHROPIC", "ANTHROPIC_KEY"]
key = None
for name in CANDIDATES:
    try:
        v = userdata.get(name)
        if v:
            key = v
            break
    except Exception:
        pass

if not key:
    raise RuntimeError(
        "Missing Anthropic API key. Add a Colab Secret named ANTHROPIC_API_KEY (or ANTHROPIC)."
    )

os.environ["ANTHROPIC_API_KEY"] = key
print("Loaded ANTHROPIC_API_KEY from Colab Secrets.")

Loaded ANTHROPIC_API_KEY from Colab Secrets.


In [12]:
from typing import TypedDict, Literal, Optional

from langchain_anthropic import ChatAnthropic
from langchain_core.messages import SystemMessage, HumanMessage

from langgraph.graph import StateGraph, START, END


class AppState(TypedDict):
    user_input: str
    route: Optional[Literal["math", "writing", "general"]]
    draft: Optional[str]
    critique: Optional[str]
    tries: int
    final: Optional[str]


llm = ChatAnthropic(
    model="claude-sonnet-4-6",
    temperature=0.3,
)


def router_node(state: AppState) -> AppState:
    msgs = [
        SystemMessage(
            content=(
                "Classify the user's request into exactly one label:\n"
                "- math: calculations, quantitative reasoning, formulas\n"
                "- writing: rewriting, polishing, summarizing, formatting text\n"
                "- general: anything else\n"
                "Return ONLY the label."
            )
        ),
        HumanMessage(content=state["user_input"]),
    ]
    resp = llm.invoke(msgs).content.strip().lower()
    route: Literal["math", "writing", "general"] = "general"
    if "math" in resp:
        route = "math"
    elif "writing" in resp:
        route = "writing"
    return {**state, "route": route}


def math_node(state: AppState) -> AppState:
    critique = state.get("critique") or ""
    msgs = [
        SystemMessage(
            content=(
                "You are a careful math tutor.\n"
                "Produce a clear solution with:\n"
                "1) Given/Goal\n"
                "2) Step-by-step reasoning\n"
                "3) Final answer\n"
                "If critique is provided, fix those issues."
            )
        ),
        HumanMessage(
            content=f"User request:\n{state['user_input']}\n\nCritique to address:\n{critique}"
        ),
    ]
    draft = llm.invoke(msgs).content
    return {**state, "draft": draft}


def writing_node(state: AppState) -> AppState:
    critique = state.get("critique") or ""
    msgs = [
        SystemMessage(
            content=(
                "You are a writing assistant.\n"
                "Follow the user's formatting requirements precisely.\n"
                "If critique is provided, fix those issues."
            )
        ),
        HumanMessage(
            content=f"User request:\n{state['user_input']}\n\nCritique to address:\n{critique}"
        ),
    ]
    draft = llm.invoke(msgs).content
    return {**state, "draft": draft}


def general_node(state: AppState) -> AppState:
    critique = state.get("critique") or ""
    msgs = [
        SystemMessage(
            content=(
                "You are a helpful assistant.\n"
                "Be accurate and follow the user's constraints.\n"
                "If critique is provided, fix those issues."
            )
        ),
        HumanMessage(
            content=f"User request:\n{state['user_input']}\n\nCritique to address:\n{critique}"
        ),
    ]
    draft = llm.invoke(msgs).content
    return {**state, "draft": draft}


def checker_node(state: AppState) -> AppState:
    draft = state.get("draft") or ""
    msgs = [
        SystemMessage(
            content=(
                "You are a strict QA checker.\n"
                "Decide whether the draft fully satisfies the user request.\n"
                "Return JSON with keys:\n"
                '{ "verdict": "ok"|"revise", "critique": "..." }\n'
                "Keep critique short and actionable."
            )
        ),
        HumanMessage(content=f"User request:\n{state['user_input']}\n\nDraft:\n{draft}"),
    ]
    resp = llm.invoke(msgs).content.strip().lower()

    verdict = "revise"
    if '"verdict"' in resp and '"ok"' in resp:
        verdict = "ok"
    elif resp.strip() == "ok":
        verdict = "ok"

    critique = ""
    if "critique" in resp:
        critique = resp.split("critique", 1)[-1].strip()[:500]

    tries = state.get("tries", 0) + 1

    if verdict == "ok" or tries >= 2:
        return {**state, "final": draft, "tries": tries, "critique": critique}
    else:
        return {**state, "tries": tries, "critique": critique, "final": None}


def route_selector(state: AppState) -> str:
    return state["route"] or "general"


def check_selector(state: AppState) -> str:
    if state.get("final"):
        return "end"
    return state["route"] or "general"


graph = StateGraph(AppState)

graph.add_node("router", router_node)
graph.add_node("math", math_node)
graph.add_node("writing", writing_node)
graph.add_node("general", general_node)
graph.add_node("checker", checker_node)

graph.add_edge(START, "router")
graph.add_conditional_edges(
    "router",
    route_selector,
    {"math": "math", "writing": "writing", "general": "general"},
)

graph.add_edge("math", "checker")
graph.add_edge("writing", "checker")
graph.add_edge("general", "checker")

graph.add_conditional_edges(
    "checker",
    check_selector,
    {"math": "math", "writing": "writing", "general": "general", "end": END},
)

app = graph.compile()


def run_once(user_input: str) -> str:
    init: AppState = {
        "user_input": user_input,
        "route": None,
        "draft": None,
        "critique": None,
        "tries": 0,
        "final": None,
    }
    out = app.invoke(init)
    return out.get("final") or "(no output)"

In [13]:
# Cell 4: run a quick demo
print(run_once("Explain LangGraph in 5 bullet points."))

Here are 5 bullet points explaining LangGraph:

• **Graph-Based Framework** – LangGraph is a library built on top of LangChain that allows developers to build stateful, multi-step AI applications by modeling workflows as graphs, where nodes represent actions or computations and edges define the flow between them.

• **Stateful Execution** – Unlike simple linear chains, LangGraph maintains a shared state across all nodes in the graph, allowing information to persist and be updated as the workflow progresses through different steps.

• **Cycles and Loops Support** – One of LangGraph's key strengths is its ability to handle cyclic graphs, enabling loops and conditional branching, which is essential for building agentic systems that need to retry, reflect, or iterate on tasks.

• **Agent and Multi-Agent Workflows** – LangGraph is particularly well-suited for building autonomous AI agents and complex multi-agent systems, where multiple AI models or tools can collaborate, hand off tasks, and

In [14]:
# Cell 5: try more prompts
tests = [
    "Rewrite this in a professional tone: I messed up the report and I need more time.",
    "Solve: If x + y = 10 and x - y = 2, find x and y.",
    "Give me a concise checklist for preparing for a coding interview.",
]
for t in tests:
    print("\n---\nPrompt:", t)
    print(run_once(t))


---
Prompt: Rewrite this in a professional tone: I messed up the report and I need more time.
"I apologize for the oversight in the report and would like to request an extension to ensure the final submission meets the required standards."

---
Prompt: Solve: If x + y = 10 and x - y = 2, find x and y.
# Solving the System of Equations

## Given Equations
$$x + y = 10 \quad \text{...(1)}$$
$$x - y = 2 \quad \text{...(2)}$$

## Solution

**Step 1: Add both equations together**

$$( x + y) + (x - y) = 10 + 2$$

$$2x = 12$$

$$x = 6$$

**Step 2: Substitute x = 6 into equation (1)**

$$6 + y = 10$$

$$y = 10 - 6$$

$$y = 4$$

## Answer

$$\boxed{x = 6, \quad y = 4}$$

## Verification
| Equation | Check |
|----------|-------|
| x + y = 10 | 6 + 4 = 10 ✅ |
| x - y = 2 | 6 - 4 = 2 ✅ |

---
Prompt: Give me a concise checklist for preparing for a coding interview.
# Coding Interview Preparation Checklist

## 📚 Data Structures
- [ ] Arrays & Strings
- [ ] Linked Lists
- [ ] Stacks & Queues
- [ ]

In [11]:
import anthropic
client = anthropic.Anthropic()
models = client.models.list()
print([m.id for m in models.data])

['claude-sonnet-4-6', 'claude-opus-4-6', 'claude-opus-4-5-20251101', 'claude-haiku-4-5-20251001', 'claude-sonnet-4-5-20250929', 'claude-opus-4-1-20250805', 'claude-opus-4-20250514', 'claude-sonnet-4-20250514', 'claude-3-7-sonnet-20250219', 'claude-3-5-haiku-20241022', 'claude-3-haiku-20240307']
